The goal of this project is to explore an analize the data captured from a postgres db.

Then, I'll pick those feautres that are more likely to predict the winners of the next event, apply data analyze and feature enginner and then save them into a mongodb.

Once the data is saved in the mongodb instance. I'll recover and apply some static stadistical models and visualization.

In [3]:
import pandas as pd

## Fetch data from Postgres

In [ ]:
master_df = pd.read_csv('../data/ufc-master.csv')
upcoming_df = pd.read_csv('../data/upcoming-event.csv')
master_df_columns = list(master_df.columns)
upcoming_df_columns = list(upcoming_df.columns)

## Explore raw data

checks if have the same columns

In [13]:
sorted(master_df_columns) == sorted(upcoming_df_columns)

True

We see the shape of each dataset

In [18]:
print('Master df shape:', master_df.shape)
print('Upcoming df shape:', upcoming_df.shape)

Master df shape: (4896, 119)
Upcoming df shape: (9, 119)


I notice thanks by shape df property that the number of columns is enourmous. 

I'll put the hat of an expert and I'll pick those columns that will help the most
to create a model that will predict the winners of the next upcomming ufc event.

If you want to see the differents columns, uncomment the next cell and execute it.

In [ ]:
# master_df_columns

## Select the most interesting features

## Clean data

## Add features

## Save the data into mongo instance

## Recover data from mongo instance

## Apply statical stadistics

## Data visualization